In [1]:
from pprint import pprint
import os
import re
import time
from time import sleep

from pynput.mouse import Button, Controller as MouseCtrlr
from pynput.keyboard import Key, Controller as KeyCtrlr
import pyautogui as pag
import keyboard as keyboard


from matplotlib import pyplot as plt
from PIL import Image
import cv2 as cv
import numpy as np

In [2]:
import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

# Helper functions

In [3]:
def imshow(img, cmap=None, size=10):
    if type(img) == str:
        img = cv.imread(img)
    elif(not type(img) == np.ndarray):
        return
    
    fig = plt.figure(figsize=(size,size))
    ax = plt.Axes(fig, [0,0,1,1])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(img, cmap)

In [4]:
def get_inverted(image):
    return cv.bitwise_not(image)

# get grayscale image
def get_grayscale(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)

#thresholding
def thresholding(image, threshold=180):
    return cv.threshold(image, threshold, 255, cv.THRESH_BINARY)[1]
    #return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    
def thresholdingAdaptive(image):
    return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)

In [5]:
kb = KeyCtrlr()

In [6]:
class Pos:
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __bool__(self):
        return True

In [7]:
def message(str):
    return pag.confirm(str)

In [8]:
globalWaitTime = 1
globalLimitTime = 5
globalWaitBeforeStart = 0.2

def getWaitAndLimit(beforeStart = -1, limit = -1, wait = -1):
    if wait == -1:
        wait = globalWaitTime
    if limit == -1:
        limit = globalLimitTime
    if beforeStart == -1:
        beforeStart = globalWaitBeforeStart
        
    return (beforeStart, limit, wait)

In [10]:
def waitForElement(path, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
        
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
            return Pos(x, y)
        except Exception as e:
            pass
            #print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element " + name)
    return False
        
#waitForElement("SokValjPatient")

In [11]:
def waitForElementDisappear(path, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
        except Exception as e:
            if (str(e) == "cannot unpack non-iterable NoneType object"):
                return True
            else:
                print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element disappear " + name)
    return False


In [12]:
def waitForOneOf(nameList, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
    
    while timePassed < limit:
        for name in nameList:
            pos = waitForElement(name, beforeStart = 0, limit = 0.01, waitTime = 0.01, warn = False)
            timePassed += 0.01
            if pos:
                return pos
            
        sleep(waitTime)
        timePassed += waitTime
        
    if warn:
        message("Timed out waiting for element list " + str(nameList))
        
    return False
    
#waitForOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"], limit=2)

In [13]:
def waitAndClickOneOf(nameList, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button = "left"):
    pos = waitForOneOf(nameList, beforeStart = beforeStart, limit = limit, waitTime = waitTime, conf = conf, warn = warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
            
        return pos
    
    return False


#waitAndClickOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"])

In [14]:
def waitAndClickElement(name, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button="left"):
    pos = waitForElement(name, beforeStart, limit, waitTime, conf, warn=warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        #print(f'Moving to {pos.x}, {pos.y}')
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
        return pos
    else:
        return False

#waitAndClickElement("SokValjPatient")

In [15]:
def waitAndClickXY(x, y, beforeStart = -1, waitTime = -1, button="left"):
    pos = Pos(x, y)
    return waitAndClick([pos], beforeStart = beforeStart, waitTime = waitTime, button=button)

def waitAndClick(positions, beforeStart = -1, waitTime = -1, button="left"):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    try:
        if type(positions) != list and type(positions) != tuple:
            positions = [positions]

        #print(f"Sleeping {beforeStart} before starting")
        sleep(beforeStart)
        for i, pos in enumerate(positions):
            if type(pos) == list or type(pos) == tuple:
                posObj = Pos(pos[0], pos[1])
            else:
                posObj = pos
            
            #print(text)
            if button == "left":
                pag.leftClick(posObj.x, posObj.y)
            elif button == "right":
                pag.rightClick(posObj.x, posObj.y)
            if i != len(positions) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndClick: " + str(e))
        return False
    

In [16]:
def waitAndType(texts, beforeStart = -1, waitTime = -1):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    try:
        if type(texts) == str:
            texts = [texts]

        sleep(beforeStart)
        for i, text in enumerate(texts):
            #print(text)
            kb.type(text)
            if i != len(texts) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndType: " + str(e))
        return False
        
        
#waitAndType(["hej", "hej2", "hej3"])
#waitAndType("hejdå")

In [17]:
def waitAndPress(key1, key2 = None, key3 = None, beforeStart = -1):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart)
    
    sleep(beforeStart)
    
    try:
        if key2 and key3:
            keyboard.press_and_release(key1+"+"+key2+"+"+key3)
        elif key2:
            pag.press_and_release(key1+"+"+key2)
        else:
            pag.press_and_release(key1)
        return True
    except Exception as e:
        pag.confirm(f"Failed wait and press {str(key1)},{str(key2)},{str(key3)}")
        return False
    

# Complex function

In [28]:
class ImgsEnum:
    def __init__(self, imagesPath):
        self.ArbetsflodeLabel = os.path.join(imagesPath, "Vaccine", "ArbetsflodeLabel.png")
        self.StangButtonText = os.path.join(imagesPath, "Vaccine", "StangButtonText.png")
        self.NyDiagnosTab = os.path.join(imagesPath, "Vaccine", "NyDiagnosTab.png")
        self.KshDropdownChoice = os.path.join(imagesPath, "Vaccine", "KshDropdownChoice.png")
        self.icd10DropdownChoice = os.path.join(imagesPath, "Vaccine", "icd10DropdownChoice.png")
        self.icdSearchInputField = os.path.join(imagesPath, "Vaccine", "icdSearchInputField.png")
        self.NyDiagnosSearchField = os.path.join(imagesPath, "Vaccine", "NyDiagnosSearchField.png")
        self.NyAtgardTab = os.path.join(imagesPath, "Vaccine", "NyAtgardTab.png")
        self.covid19AtgardChoice = os.path.join(imagesPath, "Vaccine", "covid19AtgardChoice.png")
        self.VaccinationCovid19Choice = os.path.join(imagesPath, "Vaccine", "VaccinationCovid19Choice.png")
        self.AtgardRelateradCovid19 = os.path.join(imagesPath, "Vaccine", "AtgardRelateradCovid19.png")
        self.CitrixIcon = os.path.join(imagesPath, "Vaccine", "CitrixIcon.png")
        self.LotNrLabel = os.path.join(imagesPath, "Vaccine", "LotNrLabel.png")
        self.DosNummerInputField = os.path.join(imagesPath, "Vaccine", "DosNummerInputField.png")
        self.KontraindikationNejCheckbox = os.path.join(imagesPath, "Vaccine", "KontraindikationNejCheckbox.png")
        #self. = os.path.join(imagesPath, "Vaccine", ".png")

    
IMG = ImgsEnum(os.path.join("..", "Images"))

IMG.ArbetsflodeLabel

'..\\Images\\Vaccine\\ArbetsflodeLabel.png'